# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db

Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

In [2]:
%%sql
SELECT name
FROM sqlite_master
where type = 'table'

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [3]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.


In [16]:
%%sql
-- murder in sql city on jan 15 2018

SELECT date, type, description, city
FROM crime_scene_report
WHERE date = 20180115 AND city = 'SQL City' AND type = 'murder'

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [39]:
%%sql
-- first witness last house on northwestern dr

SELECT *
FROM person
WHERE 
    address_street_name = 'Northwestern Dr'
    AND address_number = (
        SELECT MAX(address_number)
        FROM person
        WHERE address_street_name = 'Northwestern Dr'
    )

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [33]:
%%sql
-- second witness named Annabel lives on Franklin Ave

SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave' AND name LIKE '%Annabel%'

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [34]:
%%sql

SELECT *
FROM interview
WHERE person_id IN (14887, 16371)

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [66]:
%%sql

SELECT name
FROM person
LEFT JOIN drivers_license ON person.license_id = drivers_license.id
WHERE name IN (
    SELECT name
    FROM get_fit_now_check_in
    LEFT JOIN get_fit_now_member ON get_fit_now_check_in.membership_id = get_fit_now_member.id
    WHERE check_in_date = 20180109 -- was at gym on jan 9
        AND membership_status = 'gold' -- gold membership
        AND membership_id LIKE '48Z%' -- gym number starts with '48Z'
)
AND plate_number LIKE '%H42W%' -- license plate has H42W
AND gender = 'male' -- killer is a man

Running query in 'sqlite:///sql-murder-mystery.db'

name
Jeremy Bowers


### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.

In [67]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."
